## Setup

Vi definerer hjælpefunktioner til prisfastsættelse, objective-funktion, første og anden afledte.  
Disse bruges i de følgende delopgaver.

In [2]:
import sys
import os

# Gå til parent og så From Class
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), "From Class"))
from fixed_income_derivatives_E2025 import *


In [3]:
# Test at det virker
T = np.array([0.5, 1.0, 2.0, 5.0, 10.0])
R = np.array([0.05, 0.05, 0.05, 0.05, 0.05])  # 5% flad kurve

p = zcb_prices_from_spot_rates(T, R, method="continuous")
print("ZCB priser:", p)

ZCB priser: [0.97530991 0.95122942 0.90483742 0.77880078 0.60653066]


In [2]:
### Step-up ###
import numpy as np
from scipy.optimize import minimize, Bounds

def price_fixed_rate_bond_from_ytm(y, T, C):
    price = 0
    N = len(T)
    for i in range(N):
        price += C[i] / (1 + y) ** T[i]
    return price

def ytm_obj(y, pv, T, C):
    pv_hat = price_fixed_rate_bond_from_ytm(y, T, C)
    return (pv - pv_hat) ** 2

def ytm_obj_d(y, pv, T, C):
    N = len(T)
    pv_hat = price_fixed_rate_bond_from_ytm(y, T, C)
    dpv_dy = 0
    for i in range(N):
        dpv_dy += -C[i] * T[i] / (1 + y) ** (T[i] + 1)
    return -2 * (pv - pv_hat) * dpv_dy

def ytm_obj_dd(y, pv, T, C):
    N = len(T)
    pv_hat = price_fixed_rate_bond_from_ytm(y, T, C)
    dpv_dy, ddpv_ddy = 0, 0
    for i in range(N):
        dpv_dy += -C[i] * T[i] / (1 + y) ** (T[i] + 1)
        ddpv_ddy += C[i] * T[i] * (T[i] + 1) / (1 + y) ** (T[i] + 2)
    return 2 * (pv_hat - pv) * ddpv_ddy + 2 * (dpv_dy) ** 2

# inputs
R = 0.06
K = 100
T_N = 10
alpha = 0.5
pv = 98.74

N = int(T_N / alpha) + 1
T = np.array([i * alpha for i in range(N)])
C = np.zeros(N)
for i in range(N):
    C[i] += R * alpha * K
C[-1] += K


## Problem 1c

Vi finder yield to maturity numerisk ved to derivative-frie metoder:  
- **Nelder–Mead**  
- **Powell**

Begge minimerer \( SE(y) \) uden at bruge afledte.

**Nelder–Mead konvergerer til en YTM på ca. 6,7%, hvilket matcher obligationens markedspris tæt.**

In [3]:
y_init = R
args = (pv, T, C)

res_nm = minimize(ytm_obj, y_init, args=args, method="nelder-mead")
print(f"Nelder-Mead YTM: {res_nm.x[0]}")

res_pw = minimize(ytm_obj, y_init, args=args, method="powell")
print(f"Powell YTM: {res_pw.x[0]}")


Nelder-Mead YTM: 0.06696093750000001
Powell YTM: nan


/var/folders/gn/s5whzrvx7n1f8wsp2546vp040000gn/T/ipykernel_88611/761348549.py:9: RuntimeWarning: invalid value encountered in sqrt
  price += C[i] / (1 + y) ** T[i]
/var/folders/gn/s5whzrvx7n1f8wsp2546vp040000gn/T/ipykernel_88611/761348549.py:9: RuntimeWarning: invalid value encountered in power
  price += C[i] / (1 + y) ** T[i]


## Problem 1d

Vi differentierer objective-funktionen analytisk og finder udtryk for:  

- Første afledte \( SE'(y) \)  
- Anden afledte \( SE''(y) \)  

Disse bruges senere i optimeringsmetoder, der kræver gradient og Hessian.


## Problem 1e

Vi bruger **BFGS**-metoden, der kræver første afledte.  
Målet er at finde YTM hurtigere og mere præcist end med Nelder–Mead og Powell.

**BFGS giver samme resultat (~6,7%), men bruger gradientinformation, hvilket gør konvergens hurtigere og mere stabil end Nelder–Mead Metoden viser, at man ved at udnytte den analytiske afledte kan forbedre beregningen. Resultatet bekræfter, at den “sande” YTM ligger lidt over kuponrenten.**

In [4]:
res_bfgs = minimize(ytm_obj, y_init, args=args, method="BFGS", jac=ytm_obj_d)
print(f"BFGS YTM: {res_bfgs.x[0]}")


BFGS YTM: 0.06696883208690854


## Problem 1f

Vi bruger **Newton-CG**, der anvender både første og anden afledte.  
Det giver hurtigere konvergens tæt på optimum.

**Newton-CG giver også ~6,7% og anvender både første og anden afledte. Metoden er særlig effektiv tæt på optimum, fordi Hessian-information accelererer konvergensen. At resultatet er identisk med BFGS viser, at løsningen er robust på tværs af metoder.**

In [5]:
res_ncg = minimize(ytm_obj, y_init, args=args,
                   method="Newton-CG", jac=ytm_obj_d, hess=ytm_obj_dd)
print(f"Newton-CG YTM: {res_ncg.x[0]}")


Newton-CG YTM: 0.06696883165863443


## Problem 1g

Vi bruger **trust-constr** (constrained optimization).  
Her indfører vi et interval for yield:

\[
y \in [0.08, \; 0.12]
\]

og finder den bedste løsning inden for dette interval.

**Når vi indfører intervallet [0.08, 0.12], finder algoritmen en YTM helt nede ved grænsen (0.08). Dette sker, fordi den egentlige optimum (ca. 0.067) ligger uden for det tilladte område.Resultatet illustrerer, at constraints kan tvinge løsningen væk fra det reelle optimum, hvilket er nyttigt i praksis hvis man vil modellere begrænsninger i markedet.**

In [6]:
bounds = Bounds([0.08], [0.12])
res_tc = minimize(ytm_obj, y_init, args=args, method="trust-constr",
                  jac=ytm_obj_d, hess=ytm_obj_dd, bounds=bounds)
print(f"trust-constr YTM (0.08–0.12): {res_tc.x[0]}")


trust-constr YTM (0.08–0.12): 0.08000036691662532
